# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5058f62670>
├── 'a' --> tensor([[-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130]])
└── 'x' --> <FastTreeValue 0x7f5058f62e50>
    └── 'c' --> tensor([[ 0.6316,  0.6617, -0.3891,  0.0374],
                        [ 0.2490, -0.5090, -0.8779, -1.3474],
                        [ 0.5837, -0.0588,  1.4873, -0.1726]])

In [4]:
t.a

tensor([[-0.5279, -0.9637,  0.3995],
        [ 0.4365,  1.7275, -0.4130]])

In [5]:
%timeit t.a

63.9 ns ± 0.0605 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5058f62670>
├── 'a' --> tensor([[-1.3464,  1.1725,  0.4144],
│                   [-0.1977, -0.2944, -1.1320]])
└── 'x' --> <FastTreeValue 0x7f5058f62e50>
    └── 'c' --> tensor([[ 0.6316,  0.6617, -0.3891,  0.0374],
                        [ 0.2490, -0.5090, -0.8779, -1.3474],
                        [ 0.5837, -0.0588,  1.4873, -0.1726]])

In [7]:
%timeit t.a = new_value

68.8 ns ± 0.0979 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130]]),
    x: Batch(
           c: tensor([[ 0.6316,  0.6617, -0.3891,  0.0374],
                      [ 0.2490, -0.5090, -0.8779, -1.3474],
                      [ 0.5837, -0.0588,  1.4873, -0.1726]]),
       ),
)

In [10]:
b.a

tensor([[-0.5279, -0.9637,  0.3995],
        [ 0.4365,  1.7275, -0.4130]])

In [11]:
%timeit b.a

59 ns ± 0.0591 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4713, -0.3044,  0.3199],
               [ 1.0105, -1.9800, -0.6669]]),
    x: Batch(
           c: tensor([[ 0.6316,  0.6617, -0.3891,  0.0374],
                      [ 0.2490, -0.5090, -0.8779, -1.3474],
                      [ 0.5837, -0.0588,  1.4873, -0.1726]]),
       ),
)

In [13]:
%timeit b.a = new_value

501 ns ± 4.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

819 ns ± 1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 12 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 405 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 216 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5034441b50>
├── 'a' --> tensor([[[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]],
│           
│                   [[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]],
│           
│                   [[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]],
│           
│                   [[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]],
│           
│                   [[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]],
│           
│                   [[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]],
│           
│                   [[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]],
│           
│                   [[-0.5279, -0.9637,  0.3995],
│                    [ 0.4365,  1.7275, -0.4130]]])
└── 'x' --> <FastTreeValue 0x7f50344417f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 69.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5058f55310>
├── 'a' --> tensor([[-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130],
│                   [-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130],
│                   [-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130],
│                   [-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130],
│                   [-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130],
│                   [-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130],
│                   [-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130],
│                   [-0.5279, -0.9637,  0.3995],
│                   [ 0.4365,  1.7275, -0.4130]])
└── 'x' --> <FastTreeValue 0x7f50344410a0>
    └── 'c' --> tensor([[ 0.6316,  0.6617, -0.3891,  0.0374],
                        [ 0.2490, -0.5090, -0.8779, -1.3474],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 49.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.6 µs ± 47.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]],
       
               [[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]],
       
               [[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]],
       
               [[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]],
       
               [[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]],
       
               [[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]],
       
               [[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]],
       
               [[-0.5279, -0.9637,  0.3995],
                [ 0.4365,  1.7275, -0.4130]]]),
    x: Batch(
           c: tensor([[[ 0.6316,  0.6617, -0.3891,  0.0374],
                       [ 0.2490, -0.5090, -0.8779, -1.3474],
                       [ 0.5837, -0.0588,  1.4873, -0.1726]],
         

In [26]:
%timeit Batch.stack(batches)

80.6 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130],
               [-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130],
               [-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130],
               [-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130],
               [-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130],
               [-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130],
               [-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130],
               [-0.5279, -0.9637,  0.3995],
               [ 0.4365,  1.7275, -0.4130]]),
    x: Batch(
           c: tensor([[ 0.6316,  0.6617, -0.3891,  0.0374],
                      [ 0.2490, -0.5090, -0.8779, -1.3474],
                      [ 0.5837, -0.0588,  1.4873, -0.1726],
                      [ 0.6316,  0.6617, -0.3891,  0.0374],
                      [ 0.2490, -

In [28]:
%timeit Batch.cat(batches)

146 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
